In [1]:
import pandas as pd
import numpy as np
from konlpy.tag import Komoran
from konlpy.tag import Twitter
from time import time
import pickle
import os
from gensim.models import Doc2Vec
from sklearn.metrics.pairwise import cosine_similarity

In [37]:
import math

In [5]:
df1 = pd.read_csv("C:/Users/CE206-26/Desktop/크롤링구글맵스라벨링1.csv",encoding='CP949')
df2 = pd.read_csv("C:/Users/CE206-26/Desktop/구글맵스띄어쓰기삭제.csv",encoding='CP949')
df_train = df1
df_test = df2

In [9]:
df_test['라벨링'] = '0' ### test 데이터로 쓰기 위해서는 라벨링 필요 ###

In [12]:
df1['라벨링'] = df1['라벨링'].astype('str') ### 라벨이 int면 라벨이라고 인식을 못함###

In [3]:
twitter = Twitter()
def tokenizer_twitter_morphs(doc):
    return twitter.morphs(doc)

def tokenizer_twitter_noun(doc):
    return twitter.nouns(doc)

def tokenizer_twitter_pos(doc):
    return twitter.pos(doc, norm=True, stem=True)

komoran = Komoran()
def tokenizer_noun(doc):
    return komoran.nouns(doc)

def tokenizer_morphs(doc):
    return komoran.morphs(doc)

C:\Users\CE206-26\Anaconda3\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')
C:\Users\CE206-26\Anaconda3\lib\site-packages\jpype\_core.py:210: UserWarning: 
-------------------------------------------------------------------------------
Deprecated: convertStrings was not specified when starting the JVM. The default
behavior in JPype will be False starting in JPype 0.8. The recommended setting
for new code is convertStrings=False.  The legacy value of True was assumed for
this session. If you are a user of an application that reported this warning,
please file a ticket with the developer.
-------------------------------------------------------------------------------

  """)


In [6]:
### 리뷰들 토큰화 시간 오래걸림 ###

df_train['token_review'] = df_train['리뷰'].apply(tokenizer_twitter_morphs)
df_test['X_test_tokkened'] = df_test['리뷰'].apply(tokenizer_twitter_morphs)
tokens = [ t for d in df_train['token_review'] for t in d]

In [7]:
import nltk
text = nltk.Text(tokens, name='NMSC')

In [14]:
from collections import namedtuple
TaggedDocument = namedtuple('TaggedDocument', 'words tags')
tagged_train_docs = [TaggedDocument(d, c) for d, c in df_train[['token_review', '라벨링']].values]
tagged_test_docs = [TaggedDocument(d, c) for d, c in df_test[['X_test_tokkened','라벨링']].values]

In [15]:
doc_vectorizer = Doc2Vec()
doc_vectorizer.build_vocab(tagged_train_docs)
print(str(doc_vectorizer))

Doc2Vec(dm/m,d100,n5,w5,mc5,s0.001,t3)


In [16]:
start = time()
for epoch in range(10):
    doc_vectorizer.train(tagged_train_docs, total_examples=doc_vectorizer.corpus_count, epochs=doc_vectorizer.iter)
    doc_vectorizer.alpha -= 0.002 # decrease the learning rate
    doc_vectorizer.min_alpha = doc_vectorizer.alpha # fix the learning rate, no decay
end = time()
print("During Time: {}".format(end-start))

C:\Users\CE206-26\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  This is separate from the ipykernel package so we can avoid doing imports until


During Time: 17.809746026992798


In [132]:
la_num = []

for n in range(len(df2)):
    sp = df2['리뷰'][n].split()
    new_vector = doc_vectorizer.infer_vector(sp)
    similar = doc_vectorizer.docvecs.most_similar([new_vector])
    sud = []
    for n in range(6):
        sud.append(list(similar[n])[1])
    la_num.append(sud)

In [133]:
la_num

[[0.0407448410987854,
  -0.027974486351013184,
  -0.03202901780605316,
  -0.04622054845094681,
  -0.10555008053779602,
  -0.16990268230438232],
 [0.1431976556777954,
  0.09213439375162125,
  0.09176336973905563,
  0.01283235102891922,
  -0.09367397427558899,
  -0.23670822381973267],
 [0.10209497064352036,
  0.05269612744450569,
  0.051949482411146164,
  0.03156112879514694,
  -0.07023743540048599,
  -0.15316860377788544],
 [0.17982715368270874,
  0.07814883440732956,
  0.05867663770914078,
  0.020626239478588104,
  -0.016248930245637894,
  -0.12914681434631348],
 [0.21406549215316772,
  -0.004197891801595688,
  -0.009816722944378853,
  -0.04533064365386963,
  -0.06804952025413513,
  -0.08999072015285492],
 [0.13132509589195251,
  0.07504502683877945,
  0.0551382452249527,
  0.04290739446878433,
  0.018848992884159088,
  -0.12044162303209305],
 [0.07716783881187439,
  0.04061470180749893,
  0.00402488186955452,
  -0.010888062417507172,
  -0.06999479234218597,
  -0.09900608658790588],
 [

In [135]:
for n in range(len(df2)):
    la_nam.append(df2['가게명칭'][n])

In [143]:
a = np.array(la_num[50])
b = np.array(la_num[248])

In [144]:
dist = np.linalg.norm(a-b)

In [145]:
dist

0.13677071989469552